In [82]:
import requests
import json
import pandas as pd
from pandas.io.json import json_normalize

# Following code makes relevant tokens necessary to conduct TD ameritrade activities on behalf of my account
# Note refresh token is valid for 90 days. It is used to generate auth token which is only valid for 30 mins
def get_auth_token():
    refresh_token = 'sXtUVLkSa8gbLhTjgVRPJDkx0J2khZOwk6uN+q0KlnrLoNuz6Fnm893VKp76zqml0GWZ73K1Oler9JJYMNFxziNe58G+SQ5zLgqLdA3EXVO3FXIPSR74glOqGddJ2hKH9tXL1oWft5WCE6ZUpq/zjPhsJSBPi8yGx2RnSABUc7FMqTeYOUcBztjjCUl4/jgk2F076hyrrDCSBX/yakGHA09jHcbQZOOl8vyrxu/UfrjEgfxNi4YuEMKz23pcHd2L77dJRBMTugYYnFzRpHMHdGukWedlgOO4/RgQgsZ3nAKEFrSYUS9XrugrREalD2Ufo+onBbkr+N1EP+BEyinufwOkrwsebgw/5FQKB9fJzbNw/79nXeS0ZG4OHlHA3shPP6JbJOUeh2Fwev/yuhrE6sGwfcdvftzn8vFTe52KGJrBe5RapEb+n+DxkJm100MQuG4LYrgoVi/JHHvlrueqQYnI53tYFD84wI4dnFH19sX584k0XZTqmBn4M2JDQqiO4SRIMAOVtkogDmlhOHMo1xDZVUG1aDRSdnoXe3EbnfnYI8h/PfIRyBUeVolmaUDfR5KUQ7etO7BMYlfC4w0hKhbcl1HayMZLi9lgtCPqW2cjoonhrywNt+TjlwG1zwxdfBL+T2YJmyb4jGYKe3r+/LfVV/Mo73zzodDajQtkCLjbtCcGPs8SrETi73eslhHuVxhUs73PBJBhwQAnwFXy1WsgNHANr3M733dxb/kJqlSkJTTN416SeuMuu69pM8j/T7AW1+Ly/d4OVjiIhpp5ZlU9U9YrSVdSQtD9A3Tt/VHI3jIHNn0yDMyweZlwfzzHKXDVEemNXPWXu3iGj/VZqujZ3RL0ZnpNHbQDf8PX+LjkY+0e7HteljkhgZlg4X0Q5F4YySluQCE=212FD3x19z9sWBHDJACbC00B75E'
    auth_params = {'grant_type':'refresh_token', 'refresh_token':refresh_token, 'client_id':'TESTAUTH122@AMER.OAUTHAP'}
    auth_api_url = 'https://api.tdameritrade.com/v1/oauth2/token'
    return "Bearer "+ requests.post(auth_api_url, data=auth_params).json()['access_token']


# Following code will get the option chain data using the authorized token
# Parameters: call_or_put: 'CALL'/'PUT'; symbol: string with stock symbol from passed in stock list
def get_raw_option_data(symbol):
    url = 'https://api.tdameritrade.com/v1/marketdata/chains'
    strike_count = '50000000' #need to figure out how to get ALL or just use an arbitraliy huge num
    strategy = 'SINGLE'
    authorization_header = {'Authorization': get_auth_token()}
    pay = {'symbol':symbol,'strikeCount':strike_count, 'strategy':strategy, 'contractType': 'CALL'}
    return [requests.get(url, params = pay, headers = authorization_header).json()['callExpDateMap'], requests.get(url, params = pay, headers = authorization_header).json()['underlyingPrice']]


def processing(symbol):
    data = get_raw_option_data(symbol)
    agg_data = {}
    date_keys = []
    for (k,v) in data[0].items():
        temp_list = []
        for value in list(v):
            temp_list.append((value, v[value][0]['ask'], v[value][0]['symbol'])) # Line Item is StrikePrice, Premium, Symbol
        agg_data[k] = temp_list

    stock_price = data[1]
    acct_balance = 10000000000

    intermediate_data = {}
    for date in agg_data:
        for items in agg_data[date]:
            if float(items[0]) >= (1.1*stock_price):
                intermediate_data[date] = (float(items[0]), items[1], ((0.3*float(items[0])) + (float(items[0]) - stock_price))) # storing date: strike(2090 val), u(premium), y(30%strike+dolar for dollar diff (strike - stock))
                break

    target_options_list = {}
    for date in intermediate_data:
        for line_item in agg_data[date]:
            #print(line_item[1], intermediate_data[date][2])
            if line_item[1] <= min(intermediate_data[date][2], acct_balance):
                target_options_list[date] = {'110% Stock': intermediate_data[date][0], 'u': intermediate_data[date][1], 'y': intermediate_data[date][2], 'z': line_item[1], 'w': float(line_item[0]), 'symbol': line_item[2],'PROFIT': (intermediate_data[date][0] - float(line_item[0])) - (line_item[1] - intermediate_data[date][1])}
                break
                
    return pd.DataFrame(target_options_list)


def main():
    print(processing("AMZN"))

if __name__ == "__main__":
    main()
            


                2019-08-02:5     2019-08-09:12     2019-08-16:19  \
110% Stock            2132.5            2132.5              2140   
PROFIT                185.88            186.21            192.93   
symbol      AMZN_080219C1120  AMZN_080919C1120  AMZN_081619C1120   
u                       1.38              2.21              1.93   
w                       1120              1120              1120   
y                    833.805           833.805           843.555   
z                        828             828.5               829   

               2019-08-23:26     2019-08-30:33     2019-09-06:40  \
110% Stock              2140              2135              2140   
PROFIT                193.14             186.6            196.45   
symbol      AMZN_082319C1120  AMZN_083019C1120  AMZN_090619C1400   
u                       2.64               4.3              7.95   
w                       1120              1120              1400   
y                    843.555           837.055 